In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from underthesea import word_tokenize, pos_tag, sent_tokenize
import warnings
from gensim import corpora, models, similarities
import gensim
import re

In [4]:
import matplotlib.pyplot as plt 
import seaborn as sns
import glob
import os 
from tqdm.auto import tqdm
import re

# Function:

In [23]:
def recommender(view_prd:str, 
                dict_prd:corpora.Dictionary,
                tfidf_model:gensim.models.tfidfmodel.TfidfModel, 
                similarity:gensim.similarities.docsim.SparseMatrixSimilarity,
                prd_df:pd.DataFrame):
    # Convert search words into Sparse Vectors
    view_prd = view_prd.lower().split()
    kw_vector = dict_prd.doc2bow(view_prd)
    print("View product's vector:")
    print(kw_vector)

    # similarity calculation 
    sim = similarity[tfidf_model[kw_vector]]

    # print result
    list_id = []
    list_score = []
    for i in range(len(sim)):
        list_id.append(i)
        list_score.append(sim[i])

    # Create df
    df_res = pd.DataFrame({'id': list_id, 'score': list_score})

    # Get 5 highest score
    five_h_score = df_res.sort_values(by='score', ascending=False).head(6)
    print("Five highest scores:")
    print(five_h_score)
    print('IDS to list:')
    id_to_ls = five_h_score['id'].to_list()
    print(id_to_ls)

    # Find prd 
    prd_find = prd_df[prd_df.index.isin(id_to_ls)]
    prd_results = prd_find[['product_id','product_name','sub_category','price','rating']]
    final = pd.concat([prd_results,five_h_score], axis=1).sort_values(by='score', ascending=False)
    return final

# Load Product DF:

In [5]:
use_cols = ['product_id','sub_category','price','rating','clean_desc','product_name','clean_prd_name']
clean_prd_df = pd.read_csv('../DATA/clean_products.csv',usecols=use_cols)
clean_prd_df.head()

,product_id,product_name,sub_category,price,rating,clean_desc,clean_prd_name
0,190,"Áo ba lỗ thun gân ,form body tôn dáng",Áo Ba Lỗ,86250.0,4.9,áo lỗ chiều đường_phố nhiệt_đới tal fit xuất_x...,áo lỗ thun gân form body
1,191,"Áo Ba Lỗ Nam Trắng Chất Cotton Siêu Mát, Siêu Đẹp",Áo Ba Lỗ,26800.0,4.9,áo lỗ xuất_xứ việt_nam tổ_chức trách_nhiệm sản...,áo lỗ nam trắng chất_cotton siêu_mát đẹp
2,192,"Áo Ba Lỗ Nam Tyasuo chất vải co dãn mát, không...",Áo Ba Lỗ,39500.0,4.8,áo lỗ thương_hiệu tyasuo chiều áo không_thể đư...,áo lỗ nam tyasuo chất vải co_dãn mát không_xù mềm
3,193,ÁO BA LỖ HÀNG VIỆT NAM 100% COTTON,Áo Ba Lỗ,16500.0,4.8,áo lỗ chất_liệu hàng gửi hà_nội áo lỗ nam mặc ...,áo lỗ hàng việt_nam
4,194,Áo Thun Nam Thể Thao Ba Lỗ Mẫu Mới Siêu Đẹp (B...,Áo Ba Lỗ,45000.0,4.8,áo lỗ chiều áo không_thể hàng gửi hà_nội thông...,áo thun nam thể_thao lỗ mẫu mới đẹp


# Gensim

## Load Model and Product Dataframe:

In [17]:
# Load the TF-IDF model from the file
loaded_tfidf_model = models.TfidfModel.load('../DATA/Gensim/tfidf_model')

# Load the dictionary from the file
loaded_dictionary = corpora.Dictionary.load('../DATA/Gensim/corpus_dictionary')

# Load the similarity matrx 
loaded_similarity_matrix  = similarities.SparseMatrixSimilarity.load("../DATA/Gensim/similarity_matrix.index")

## Test Fuction:

In [18]:
text_prd = clean_prd_df.loc[100,'clean_prd_name']
text_prd

'áo thun nam lỗ lông_vũ_thoáng mát vải_sợi tre tổng_hợp đẹp'

In [24]:
results_df = recommender(view_prd=text_prd,
                dict_prd=loaded_dictionary,
                tfidf_model=loaded_tfidf_model,
                similarity=loaded_similarity_matrix,
                prd_df=clean_prd_df)

View product's vector:
[(3, 1), (4, 1), (5, 1), (7, 1), (10, 1), (14, 1), (290, 1), (291, 1), (292, 1), (293, 1)]
Five highest scores:
          id     score
100      100  1.000000
16256  16256  0.659966
7073    7073  0.640037
16325  16325  0.578909
151      151  0.474653
150      150  0.465541
IDS to list:
[100, 16256, 7073, 16325, 151, 150]


In [25]:
results_df

,product_id,product_name,sub_category,price,rating,id,score
100,19100,Áo thun nam ba lỗ TN195 lông vũ thoáng mát vải...,Áo Ba Lỗ,47170.0,4.7,100,1.000000
16256,233922,ÁO ĐÔI ÁO CẶP SO CUTE,Đồ Bộ,35000.0,5.0,16256,0.659966
7073,18689,Áo Polo Nam ELEGANCE ATINO chữ nổi tinh tế vải...,Áo,129000.0,4.9,7073,0.640037
16325,233991,⚡️ GIÁ RẺ NHẤT ⚡️ Đồ hể thao Nam - quần nỉ phố...,Đồ Bộ,52000.0,4.8,16325,0.578909
151,19151,Áo thun nam ba lỗ TN102 sát nách nhiều màu vải...,Áo Ba Lỗ,68085.0,4.7,151,0.474653
150,19150,Áo phông nam ba lỗ thể thao SPORT TN109 nhiều ...,Áo Ba Lỗ,32400.0,4.8,150,0.465541


## Apply Cosine Similarity

In [56]:
final_df = clean_prd_df.reset_index().rename(columns={'index':'item_id'})
final_df.head()

,item_id,product_id,product_name,sub_category,price,rating,clean_desc,clean_prd_name
0,0,190,"Áo ba lỗ thun gân ,form body tôn dáng",Áo Ba Lỗ,86250.0,4.9,áo lỗ chiều đường_phố nhiệt_đới tal fit xuất_x...,áo lỗ thun gân form body
1,1,191,"Áo Ba Lỗ Nam Trắng Chất Cotton Siêu Mát, Siêu Đẹp",Áo Ba Lỗ,26800.0,4.9,áo lỗ xuất_xứ việt_nam tổ_chức trách_nhiệm sản...,áo lỗ nam trắng chất_cotton siêu_mát đẹp
2,2,192,"Áo Ba Lỗ Nam Tyasuo chất vải co dãn mát, không...",Áo Ba Lỗ,39500.0,4.8,áo lỗ thương_hiệu tyasuo chiều áo không_thể đư...,áo lỗ nam tyasuo chất vải co_dãn mát không_xù mềm
3,3,193,ÁO BA LỖ HÀNG VIỆT NAM 100% COTTON,Áo Ba Lỗ,16500.0,4.8,áo lỗ chất_liệu hàng gửi hà_nội áo lỗ nam mặc ...,áo lỗ hàng việt_nam
4,4,194,Áo Thun Nam Thể Thao Ba Lỗ Mẫu Mới Siêu Đẹp (B...,Áo Ba Lỗ,45000.0,4.8,áo lỗ chiều áo không_thể hàng gửi hà_nội thông...,áo thun nam thể_thao lỗ mẫu mới đẹp


In [68]:
final_df.loc[final_df['item_id'] == 1,'product_name'].values[0]

'Áo Ba Lỗ Nam Trắng Chất Cotton Siêu Mát, Siêu Đẹp'

In [69]:
def recommend_cosine(prd_id,
                     df,
                     cosine_array, num = 5):
    text =  df.loc[df['item_id'] == prd_id,'product_name'].values[0]
    print(f'Recommending {num} products similar to ***{text}***')
    recs = cosine_array[prd_id][:num]
    for rec in recs:
        #print(rec[1])
        print(f"Recommnended:\tItem ID: {rec[1]}, {df.loc[df['item_id'] == rec[1],'product_name'].values[0]}, (score: {rec[0]})\n")

In [71]:
recommend_cosine(prd_id=2001,df=final_df,cosine_array=results,num=4)

Recommending 4 products similar to ***áo sọc nam***
Recommnended:	Item ID: 474, áo 3 lỗ nam sọc ngang Mã: NB073, (score: 0.7646320602976147)

Recommnended:	Item ID: 41193, Áo Sơ Mi Nam Dài Tay Kẻ Sọc Thời Trang, (score: 0.8196531776925334)

Recommnended:	Item ID: 35176, Quần thể thao kẻ sọc nam 1 sọc - 2 sọc - 3 sọc, (score: 0.8400904703451335)

Recommnended:	Item ID: 18173, Áo Thun Nam Kẻ Sọc Thời Trang Cy33, (score: 0.8549251132460183)

